# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
import os
#import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display


In [ ]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

SYSTEM_MESSAGE = "Provide answers to coding and LLM-related questions. Respond in formatted Markdown where appropriate."

In [ ]:
# set up environment
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
assert openai_api_key and openai_api_key.startswith('sk-proj-') and len(openai_api_key) > 10


In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

def process_stream_response(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

def call_chatgpt(system_prompt, user_prompt):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ],
        stream=True
    )
    return process_stream_response(stream)

response = call_chatgpt(SYSTEM_MESSAGE, question)


In [ ]:
# Get Llama 3.2 to answer
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def call_llama(system_prompt, user_prompt):

    stream = openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ],
        stream=True
    )
    return process_stream_response(stream)

response = call_llama(SYSTEM_MESSAGE, question)
